In [22]:
import pandas as pd
import re
import preprocessor as p
import string
import emoji
import unicodedata
from nltk.corpus import stopwords

### Importing dataset

In [23]:
ds = pd.read_csv('..\\Datasets\\Training Datasets\\italian_dataset.csv', usecols=[
    'tweet_text', 'sentiment'], engine='python')

In [24]:
ds.head(10)

,tweet_text,sentiment
0,"@juventusfc @G_Higuain Auguri pipita,sempre co...",NEUTRAL
1,@realvarriale @massimozampini @KMbappe @G_Higu...,NEUTRAL
2,@OfficialASRoma @D_10Perotti @Hyundai_Italia S...,NEUTRAL
3,@CarloRienzi @Cristiano @juventusfc Populismo ...,NEUTRAL
4,@OfficialASRoma @OfficialRadja @leonsfdo non c...,NEUTRAL
5,@BAILALASAMBA @Milanistiblog @ECavaniOfficial ...,NEUTRAL
6,@95Luk @calciomercatoit @G_Higuain È gli altri...,NEUTRAL
7,"@SergioRamos non sei un cane, sei una piccola ...",NEUTRAL
8,@DSportiva Ma @PauDybala_JR non si tocca.. res...,NEUTRAL
9,@antoniusromano @Cristiano @antoniusromano ma ...,NEUTRAL


### Preprocessor and emoji

In [25]:
# Setting preprocesor options
p.set_options(p.OPT.URL,p.OPT.MENTION)

In [26]:
# Function for cleaning with preprocessor
# It removes URLs and mentions
def preprocess_tweet(row):
    text = row['tweet_text']
    text = p.clean(text)
    return text

In [27]:
# Demojize
# Replaces emojis with text
def demojize(row):
    text = row['tweet_text']
    text = emoji.demojize(text)
    return text

In [28]:
# Applying previously defined functions
ds['tweet_text'] = ds.apply(preprocess_tweet, axis=1)
ds['tweet_text'] = ds.apply(demojize, axis=1)

In [29]:
ds.head(10)

,tweet_text,sentiment
0,"Auguri pipita,sempre con noi",NEUTRAL
1,Nano puttano,NEUTRAL
2,Stasera bravissimi tutti! Una menzione per il ...,NEUTRAL
3,Populismo di basso spessore,NEUTRAL
4,non ce posso pensa’ non giochiamo #Radjanonsit...,NEUTRAL
5,- stop! Altrimenti tanto vale prendere Immobik...,NEUTRAL
6,È gli altri 50 dove li prendi?,NEUTRAL
7,"non sei un cane, sei una piccola cagna, spaven...",NEUTRAL
8,Ma non si tocca.. restasse da noi tutta la vita,NEUTRAL
9,ma ritornerete con il nostro canale ufficiale?,NEUTRAL


### Re

In [30]:
# Deleting duplicate tweets
ds = ds.drop_duplicates()

# Removing links
ds.tweet_text = ds.tweet_text.replace("https?://[\w/%-.]*", " ", regex=True)

# Removing the space at the beginning
ds.tweet_text = ds.tweet_text.replace('^ ', '', regex=True)
# Removing the space at the end
ds.tweet_text = ds.tweet_text.replace(' $', '', regex=True)
# Removing the spaces in excess
ds.tweet_text = ds.tweet_text.replace('\s+', ' ', regex=True)

ds.tweet_text = ds.tweet_text.replace('^', ' ', regex=True)
ds.tweet_text = ds.tweet_text.replace('$', ' ', regex=True)
ds.tweet_text = ds.tweet_text.replace(
    '/(?:https?|ftp):\/\/[\n\S]+/g', '', regex=True)
ds.tweet_text = ds.tweet_text.replace('#', '', regex=True)

# To lowercase
ds.tweet_text = ds.tweet_text.apply(
    lambda x: x.lower())

# Removing the space at the beginning and at the end
ds.tweet_text = ds.tweet_text.apply(lambda x: x.strip())

# Removing empty tweets
ds = ds[ds.tweet_text != '']

In [31]:
ds.head(10)

,tweet_text,sentiment
0,"auguri pipita,sempre con noi",NEUTRAL
1,nano puttano,NEUTRAL
2,stasera bravissimi tutti! una menzione per il ...,NEUTRAL
3,populismo di basso spessore,NEUTRAL
4,non ce posso pensa’ non giochiamo radjanonsito...,NEUTRAL
5,- stop! altrimenti tanto vale prendere immobik...,NEUTRAL
6,è gli altri 50 dove li prendi?,NEUTRAL
7,"non sei un cane, sei una piccola cagna, spaven...",NEUTRAL
8,ma non si tocca.. restasse da noi tutta la vita,NEUTRAL
9,ma ritornerete con il nostro canale ufficiale?,NEUTRAL


### Stopwords removal

In [32]:
# Defining stopwords and adding a space before and after to exclude the case
# in which the stopword is cointained in a word
words = set(stopwords.words('italian'))
stopwords = [' ' + x + ' ' for x in words]

# Removing stopwords
for word in stopwords:
    ds.tweet_text = ds.tweet_text.replace(word, ' ', regex=True)

In [33]:
ds.head(10)

,tweet_text,sentiment
0,"auguri pipita,sempre noi",NEUTRAL
1,nano puttano,NEUTRAL
2,stasera bravissimi tutti! menzione capitano lu...,NEUTRAL
3,populismo basso spessore,NEUTRAL
4,non ce posso pensa’ giochiamo radjanonsitocca ...,NEUTRAL
5,- stop! altrimenti tanto vale prendere immobik...,NEUTRAL
6,è altri 50 prendi?,NEUTRAL
7,"non cane, piccola cagna, spaventata! vorrei pi...",NEUTRAL
8,ma tocca.. restasse tutta vita,NEUTRAL
9,ma ritornerete canale ufficiale?,NEUTRAL


### Further cleaning

In [34]:
# Removing usernames in case preprocessor didn't work
def remove_mentions(text):
    return re.sub(r"(?:\@|https?\://)\S+", "", text)


# Remove punctuation in case Re missed something
def remove_punctuations(text):
    for punctuation in string.punctuation:
        text = text.replace(punctuation, ' ')
    return text

In [35]:
# Changing string to numeric values
ds['sentiment'] = ds['sentiment'].replace(
    ['NEGATIVE', 'NEUTRAL', 'POSITIVE'], [-1, 0, 1])

# Removing mentions
ds['tweet_text'] = ds['tweet_text'].apply(remove_mentions)

# Removing punctuation
ds['tweet_text'] = ds['tweet_text'].apply(remove_punctuations)

# Removing the spaces in excess
ds['tweet_text'] = ds['tweet_text'].replace(
    '\s+', ' ', regex=True)

# Removing mixed sentiments
ds = ds[ds.sentiment != 'MIXED']

In [36]:
ds.head(10)

,tweet_text,sentiment
0,auguri pipita sempre noi,0
1,nano puttano,0
2,stasera bravissimi tutti menzione capitano luc...,0
3,populismo basso spessore,0
4,non ce posso pensa’ giochiamo radjanonsitocca ...,0
5,stop altrimenti tanto vale prendere immobiken...,0
6,è altri 50 prendi,0
7,non cane piccola cagna spaventata vorrei pisci...,0
8,ma tocca restasse tutta vita,0
9,ma ritornerete canale ufficiale,0


In [45]:
# Remove accents in order to mitigate weird charachters in the final result
def strip_accents(row):
    text = row['tweet_text']
    try:
        text = unicode(text, 'utf-8')
    except NameError: # unicode is a default on python 3 
        pass

    text = unicodedata.normalize('NFD', text)\
           .encode('ascii', 'ignore')\
           .decode("utf-8")

    return text

In [47]:
# Removing accent
ds['tweet_text'] = ds.apply(strip_accents, axis=1)

In [53]:
df_sorted = ds.sort_values(by=['sentiment'], ascending=True)

In [54]:
ds.head(10)

,tweet_text,sentiment
0,auguri pipita sempre noi,0
1,nano puttano,0
2,stasera bravissimi tutti menzione capitano luc...,0
3,populismo basso spessore,0
4,non ce posso pensa giochiamo radjanonsitocca p...,0
5,stop altrimenti tanto vale prendere immobiken...,0
6,e altri 50 prendi,0
7,non cane piccola cagna spaventata vorrei pisci...,0
8,ma tocca restasse tutta vita,0
9,ma ritornerete canale ufficiale,0


In [50]:
df_sorted.to_csv('..\\Datasets\\Training Datasets\\cleanDataset.csv')